In [8]:
# NAME- KARTIKAY SINGHAL
# ROLL NO- 231030026

# feature extraction is done in other code ,the csv file generated from that code is used here for training
# classification of pavement using ANN using Leaky-Relu activation function for hidden layers and for output layer sigmoid activation function
# leaky-relu to avoid learning slowdown and dead-neurons 
# importing libraries
# 1) standard scalar  for scaling of features.
# 2) labelencoder for converting classes into binary. 
# 3) from keras using sequential model,which allows to add layers one by one in stacks
# 4) keras ANN library adopted,for creating input,hidden and output layers dense module is imported  for fully connnected layers.
# 5) from Keras.layers, dropout function is taken for regularization of ANN training algorithm and preventing of overfitting.
# 6) from Keras.layers, BatchNormalization function is taken for normalizing each existing layers inputs activations for convergence in neural network 
# 7) from keras importing modelcheckpoint.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from keras.callbacks import ModelCheckpoint
import re

# Natural sorting helper function to correctly order string with numerical values.
def natural_key(string):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string)]

# Step 1: Load the training dataset(700 images examples) containing 34 features of images.
data = pd.read_csv('raveling_image_features_mahotas.csv')
# Step 2: Extracting  features and target labels from training file 
X = data.iloc[:, :-1].values  # Features (34 features)
y = data.iloc[:, -1].values    # Target labels (ravelling-class or non-ravelling-class)

# Step 3: Convert categorical labels to binary(0 or 1)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)   # Convert to 0(non-raveling) and 1 (raveling)

# Step 4: Randomize and split the training images dataset(for training 80% & testing 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Step 5: Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 6: Build the improved ANN model for adding  layer by layer in stack
model = Sequential()

# Input layer and first hidden layer
# Add a dense layer with 128 neurons and Leaky-RelU activation taking input from training data.
model.add(Dense(128, input_dim=X_train.shape[1]))  # 34 input features
model.add(LeakyReLU(alpha=0.1))  # Add Leaky ReLU activation  with alpha=0.1
model.add(BatchNormalization()) # Applying batch-normalization to normalize and speed-up by normalizing output of previous layer.
model.add(Dropout(0.3)) # Adds a dropout layer to set 30% neurons to 0 to reduce overfitting


# Second hidden layer with Leaky ReLU
# Add a dense layer with 64 neurons and Leaky-RelU activation taking actuivations of first hidden layer
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1)) # Add Leaky ReLU activation  with alpha=0.1
model.add(BatchNormalization()) # Applying batch-normalization to normalize and speed-up by normalizing output of previous layer.
model.add(Dropout(0.3)) # Adds a dropout layer to set 30% neurons to 0 to reduce overfitting

# Third hidden layer with Leaky ReLU
# Add a dense layer with 32 neurons and Leaky-RelU activation taking actuivations of second hidden layer
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))  # Add Leaky ReLU activation
model.add(BatchNormalization())  # Applying batch-normalization to normalize and speed-up by normalizing output of previous layer.
model.add(Dropout(0.2)) # Adds a dropout layer to set 20% neurons to 0 to reduce overfitting

# Output layer
# produces 1 output neuron with sigmoid activation function for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model by loss function (cross-entropy), optimizer and accuracy as indicator of its performance
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 7: Set up ModelCheckpoint to save the weights of best iteration based on training accuracy
checkpoint = ModelCheckpoint('best_model.keras', monitor='accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model with 150 epochs using 16 samples per batch having 35 batches,saving best weights during training
model.fit(X_train, y_train, epochs=150, batch_size=16, verbose=1, callbacks=[checkpoint])

# Step 8: Load the best model weights
model.load_weights('best_model.keras')

# Step 9: Predict labels on the test set using the best model weights
y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary

# Print classification report and accuracy
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Step 10: Loading the external test dataset containing features of 300 test-images given in assignment
data_test = pd.read_csv('test-raveling_image_features_mahotas.csv')


# Step 11: Sort the test dataset by filenames in way that comes in the format of first 1.jpg second 2.jpg etc. using natural sorting
# done for taking output and saving in csv file in submission file format
data_test['filename'] = data_test['filename'].apply(str)
data_test_sorted = data_test.sort_values(by='filename', key=lambda x: x.map(natural_key))  # Natural sort

# Step 12: Extracting  features from the sorted test dataset (features start from the second column of dataset)
X_test_features = data_test_sorted.iloc[:, 1:].values  # Features excluding filename

# Step 13: Scale the test features using the same scaler fit as on the training data
X_test_scaled = scaler.transform(X_test_features)

# Step 14: Predict the labels for the test dataset
y_test_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")   # Converting probabilities to binary by adopting threshold of 0.5

# Step 15: Create a DataFrame for output
output_df = pd.DataFrame({
    'filename': data_test_sorted['filename'].values,  # Use sorted filenames
    'class': y_test_pred.flatten()  # Flatten to get a 1D array
})

# Step 16: Convert binary labels back to class names
output_df['class'] = output_df['class'].map({1: 'Raveling', 0: 'Non_raveling'})  # Map 1 -> Raveling, 0 -> Non_raveling

# Step 17: Save the predictions to a CSV file in submission format with file name given 
output_csv_path = 'final_submission_try_predicted_labels_new_16.csv'
output_df.to_csv(output_csv_path, index=False)

print(f"Predictions saved to {output_csv_path}")


C:\Users\kartikey singhal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\kartikey singhal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/150
18/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4999 - loss: 0.8936 
Epoch 1: accuracy improved from -inf to 0.69464, saving model to best_model.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5869 - loss: 0.7703
Epoch 2/150
16/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7275 - loss: 0.5919 
Epoch 2: accuracy improved from 0.69464 to 0.78036, saving model to best_model.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7583 - loss: 0.5471
Epoch 3/150
34/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7761 - loss: 0.4553
Epoch 3: accuracy improved from 0.78036 to 0.80000, saving model to best_model.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7774 - loss: 0.4542
Epoch 4/150
22/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8033 - loss: 0.4289 
Epoch 4: accuracy improved from 0.80000 to 0.81964, saving model to best_model.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8096 - loss: 0.4151
Epoch 5/150
29/35 ━━━━━━━━━━